In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip "/content/drive/MyDrive/dataset.zip" -d "/content/drive/MyDrive"

In [14]:
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [3]:
image_directory ='/content/drive/MyDrive/dataset/'

no_tumor_images = os.listdir(image_directory+ 'no/')
yes_tumor_images = os.listdir(image_directory+ 'yes/')

# print(no_tumor_images)
# print(yes_tumor_images)
# path = 'no0.jpg'
# print(path.split('.')[1])




In [15]:
dataset=[]
label=[]

INPUT_SIZE=64

In [16]:
for i , image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'no/'+image_name)
        image=Image.fromarray(image,'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)


# print(len(datasets))
# print(len(labels))

In [17]:
for i , image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'yes/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)


In [18]:
dataset=np.array(dataset)
label=np.array(label)

In [19]:
x_train, x_test, y_train, y_test=train_test_split(dataset, label, test_size=0.2, random_state=0)

In [20]:
x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)

In [21]:
y_train=to_categorical(y_train , num_classes=2)
y_test=to_categorical(y_test , num_classes=2)

In [22]:
# Model Building
# 64,64,3

model=Sequential()

model.add(Conv2D(32, (3,3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, (3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

In [26]:
# Binary CrossEntropy= 1, sigmoid
# Categorical Cross Entryopy= 2 , softmax

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 activation (Activation)     (None, 62, 62, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 29, 29, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                    

In [27]:
model.fit(x_train, y_train, 
batch_size=16, 
verbose=1, epochs=10, 
validation_data=(x_test, y_test),
shuffle=False)

Epoch 1/10
150/150 [==============================] - 3s 14ms/step - loss: 0.0222 - accuracy: 0.9921 - val_loss: 0.0869 - val_accuracy: 0.9783
Epoch 2/10
150/150 [==============================] - 2s 12ms/step - loss: 0.0192 - accuracy: 0.9962 - val_loss: 0.1225 - val_accuracy: 0.9600
Epoch 3/10
150/150 [==============================] - 2s 12ms/step - loss: 0.0104 - accuracy: 0.9975 - val_loss: 0.1128 - val_accuracy: 0.9733
Epoch 4/10
150/150 [==============================] - 2s 12ms/step - loss: 0.0127 - accuracy: 0.9967 - val_loss: 0.1313 - val_accuracy: 0.9733
Epoch 5/10
150/150 [==============================] - 2s 12ms/step - loss: 0.0432 - accuracy: 0.9846 - val_loss: 0.1908 - val_accuracy: 0.9550
Epoch 6/10
150/150 [==============================] - 2s 12ms/step - loss: 0.0229 - accuracy: 0.9917 - val_loss: 0.1072 - val_accuracy: 0.9800
Epoch 7/10
150/150 [==============================] - 2s 12ms/step - loss: 0.0050 - accuracy: 0.9987 - val_loss: 0.1153 - val_accuracy: 0.9817

In [25]:
model.save('BrainTumor10EpochsCategorical.h5')

test

In [28]:
import cv2
from keras.models import load_model
from PIL import Image
import numpy as np

In [34]:
model=load_model('BrainTumor10EpochsCategorical.h5')

image=cv2.imread('/content/pred5.jpg')

img=Image.fromarray(image)

img=img.resize((64,64))

img=np.array(img)

input_img=np.expand_dims(img, axis=0)

predict=model.predict(input_img)
result =np.argmax(predict,axis=1)
print(result)

[1]
